In [1]:
# !pip install selenium
# !pip install chromedriver-autoinstaller


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [91]:
import chromedriver_autoinstaller # chrome driver 자동 설치 라이브러리
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [97]:
# chrome driver를 자동으로 설치함
chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions() # Browser 세팅하기
options.add_argument('lang=ko_KR') # 사용언어 한국어
options.add_argument('disable-gpu') # 하드웨어 가속 안함
# options.add_argument('headless') # 창 숨기기

# 브라우저 세팅
driver = webdriver.Chrome(options=options)

In [98]:
# 브라우저에 URL 호출하기
target_url='https://new.cretop.com/ET/SS/ETSS071M1?h=1683949489638'
driver.get(url=target_url)

In [99]:
try:
    close_btn_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button[1]'
    search_btn_raw_xpath ='//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button[2]'

except Exception:
    search_btn_raw_xpath ='//*[@id="app"]/div[1]/div[1]/div/div/div[1]/div/div/div/div/div[2]/button' 


webEl_search_btn = driver.find_element(By.XPATH,search_btn_raw_xpath)

In [100]:
print(webEl_search_btn.text)
webEl_search_btn.click()

조회하기


In [125]:
class ResultItem():
    def __init__(self, title:str, info:list[str]):
        self.title = title
        self.infos = []
        self.infos.append(info)
    def __init__(self):
        self.title = ''
        self.infos = []

    def setTitle(self,title):
        self.title = title

    def setInfo(self, info:list[str]):
        self.infos.append(info)

In [122]:
webEl_search_result_list = driver.find_element(By.CLASS_NAME, 'search-result__list')
webEl_search_result_items = webEl_search_result_list.find_elements(By.TAG_NAME, 'li')

'''
<button>
<ul class='search-info-list'>
<ul class='search-info-list'>
<ul class='btn__list'>
'''

result_items:list[ResultItem] = []

for search_result_item in webEl_search_result_items:
    try:
        result_item = ResultItem()
        title = search_result_item.find_element(By.TAG_NAME, 'button').text
        result_item.setTitle(title)
        print(title)
        info_list = search_result_item.find_elements(By.CLASS_NAME, 'search-info-list')
        for info_items in info_list:
            info_item = info_items.find_elements(By.TAG_NAME, 'li')
            for li in info_item:
                info_title = li.find_element(By.CLASS_NAME, 'list-tit')
                info_info = li.find_element(By.CLASS_NAME, 'list-info')
                print(info_title.text, info_info.text)

                result_item.setInfo([info_title.text, info_info.text])
        result_items.append(result_item)
    except Exception:
        pass

현대자동차(주)
대표자명 정의선
기업상태 정상
기업유형/형태 유가증권시장
사업자번호 101-81-0****
법인번호 110111-*******
산업분류 승용차 및 기타 여객용 자동차 제조업
주소 서울 서초구 헌릉로
메리츠증권(주)
대표자명 최알렉산더희문
기업상태 정상
기업유형/형태 외감
사업자번호 116-81-2****
법인번호 110111-*******
산업분류 증권 중개업
주소 서울 영등포구 국제금융로
지에스칼텍스(주)
대표자명 허세홍
기업상태 정상
기업유형/형태 외감
사업자번호 116-81-1****
법인번호 110111-*******
산업분류 원유 정제처리업
주소 서울 강남구 논현로
(주)하나은행
대표자명 이승열
기업상태 정상
기업유형/형태 외감
사업자번호 202-81-1****
법인번호 110111-*******
산업분류 국내은행
주소 서울 중구 을지로
에스케이에너지(주)
대표자명 조경목
기업상태 정상
기업유형/형태 외감
사업자번호 101-86-6****
법인번호 110111-*******
산업분류 원유 정제처리업
주소 서울 종로구 종로
(주)국민은행
대표자명 이재근
기업상태 정상
기업유형/형태 외감
사업자번호 201-81-6****
법인번호 110111-*******
산업분류 국내은행
주소 서울 영등포구 국제금융로8길
기아(주)
대표자명 최준영
기업상태 정상
기업유형/형태 유가증권시장
사업자번호 119-81-0****
법인번호 110111-*******
산업분류 승용차 및 기타 여객용 자동차 제조업
주소 서울 서초구 헌릉로
에쓰대시오일(주)
대표자명 후세인에이알-카타니
기업상태 정상
기업유형/형태 유가증권시장
사업자번호 116-81-3****
법인번호 110111-*******
산업분류 원유 정제처리업
주소 서울 마포구 백범로
(주)우리은행
대표자명 이원덕
기업상태 정상
기업유형/형태 외감
사업자번호 201-81-0****
법인번호 110111-*******
산업분류 국내은행
주소 서울 중구 소공로
삼성생명보험(주)
대표자명 전영묵
기업상태 정상
기

In [123]:
for item in result_items:
    print(item.title, item.infos)

현대자동차(주) [['대표자명', '정의선'], ['기업상태', '정상'], ['기업유형/형태', '유가증권시장'], ['사업자번호', '101-81-0****'], ['법인번호', '110111-*******'], ['산업분류', '승용차 및 기타 여객용 자동차 제조업'], ['주소', '서울 서초구 헌릉로']]
메리츠증권(주) [['대표자명', '최알렉산더희문'], ['기업상태', '정상'], ['기업유형/형태', '외감'], ['사업자번호', '116-81-2****'], ['법인번호', '110111-*******'], ['산업분류', '증권 중개업'], ['주소', '서울 영등포구 국제금융로']]
지에스칼텍스(주) [['대표자명', '허세홍'], ['기업상태', '정상'], ['기업유형/형태', '외감'], ['사업자번호', '116-81-1****'], ['법인번호', '110111-*******'], ['산업분류', '원유 정제처리업'], ['주소', '서울 강남구 논현로']]
(주)하나은행 [['대표자명', '이승열'], ['기업상태', '정상'], ['기업유형/형태', '외감'], ['사업자번호', '202-81-1****'], ['법인번호', '110111-*******'], ['산업분류', '국내은행'], ['주소', '서울 중구 을지로']]
에스케이에너지(주) [['대표자명', '조경목'], ['기업상태', '정상'], ['기업유형/형태', '외감'], ['사업자번호', '101-86-6****'], ['법인번호', '110111-*******'], ['산업분류', '원유 정제처리업'], ['주소', '서울 종로구 종로']]
(주)국민은행 [['대표자명', '이재근'], ['기업상태', '정상'], ['기업유형/형태', '외감'], ['사업자번호', '201-81-6****'], ['법인번호', '110111-*******'], ['산업분류', '국내은행'], ['주소', '서울 영등포구 국제금융로8길']]
기아(주) [['대표자명', '

In [170]:
index_div_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div'
index_div_xpath = driver.find_element(By.XPATH,index_div_raw_xpath)

In [178]:
webEl_index_btn = index_div_xpath.find_element(By.TAG_NAME, 'ul').find_elements(By.TAG_NAME,'li')
for index_btn in webEl_index_btn:
    try:
        current_index = int(index_btn.find_element(By.CLASS_NAME, 'on').text) % 10
        print('on: ', current_index)
    except Exception:
        print('off: ', index_btn.text)


off:  11
off:  12
off:  13
off:  14
off:  15
off:  16
off:  17
off:  18
off:  19
on:  0


In [179]:
# //*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[2]/button # sec btn
# //*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[3]/button # 3th btn

def click_next_index(current_index):
    next_index = str(current_index+1)
    print("current index: ",current_index)
    print("next index: ",next_index)
    if current_index == '0':
        print('마지막 번호')
        next_index_btn_xpath = driver.find_element(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/button[3]')
    else:
        next_index_btn_raw_xpath = '//*[@id="app"]/div[1]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/ul/li[{}]/button'.format(next_index)
        next_index_btn_xpath = driver.find_element(By.XPATH,next_index_btn_raw_xpath)
    next_index_btn_xpath.click()

click_next_index(current_index)

current index:  0
next index:  1


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (-128, 8333)
  (Session info: chrome=113.0.5672.92)
Stacktrace:
0   chromedriver                        0x000000010324a6c8 chromedriver + 4974280
1   chromedriver                        0x0000000103241a33 chromedriver + 4938291
2   chromedriver                        0x0000000102dfece6 chromedriver + 470246
3   chromedriver                        0x0000000102e4a533 chromedriver + 779571
4   chromedriver                        0x0000000102e47d5f chromedriver + 769375
5   chromedriver                        0x0000000102e45184 chromedriver + 758148
6   chromedriver                        0x0000000102e441de chromedriver + 754142
7   chromedriver                        0x0000000102e36681 chromedriver + 697985
8   chromedriver                        0x0000000102e669c2 chromedriver + 895426
9   chromedriver                        0x0000000102e35e61 chromedriver + 695905
10  chromedriver    

In [ ]:
# 브라우저 탭 닫기
driver.close()
# 브라우저 종료하기 (탭 모두 종료)
driver.quit()